In [6]:
import numpy as np
def ELS(r,q1,q2,sigma1,sigma2,rho,M,n):
    value_list=np.zeros(M)
    case_list=np.zeros(M)
    for j in range(M): ##M번 만큼 돌림
        X1=np.zeros(n*4+1)
        X1[0]=1
        X2=np.zeros(n*4+1)
        X2[0]=1
        for i in range(len(X1)-1):#시계열 생성
            T=1/(n*2)
            x1=np.random.normal(0,1)
            x2=np.random.normal(0,1)

            e1=x1
            e2=rho*x1+x2*np.sqrt(1-pow(rho,2))
            X1[i+1]=X1[i]/np.exp((q1-r-pow(sigma1,2)/2)*T+sigma1*e1*np.sqrt(T))
            X2[i+1]=X2[i]/np.exp((q2-r-pow(sigma2,2)/2)*T+sigma2*e2*np.sqrt(T))
        profit_flag=1
        for i in range(len(X1)-1): ##기간중 40퍼 이상 떨어진 경우가 있는 경우
            if X1[i]<0.6 or X2[i]<0.6:
                profit_flag=0
                break
        case=0
        
        ########60퍼 이하로 안떨어진 경우
        if profit_flag==1:
            if X1[n*1]>=0.85 and X2[n*1]>=0.85:
                value=100*pow(1+0.125,0.5)/pow(1+r,0.5)
                case=1
            elif X1[n*2]>=0.80 and X2[n*2]>=0.80:
                value=100*pow(1+0.125,1)/pow(1+r,1)
                case=2
            elif X1[n*3]>=0.75 and X2[n*3]>=0.75:
                value=100*pow(1+0.125,1.5)/pow(1+r,1.5)
                case=3
            elif X1[n*4]>=0.70 and X2[n*4]>=0.70:
                value=100*pow(1+0.125,2)/pow(1+r,2)
                case=4
            elif X1[n*4]<=0.70 or X2[n*4]<=0.70:
                value=100/pow(1+r,2)
                case=5
        ########60퍼 이하로 떨어진 경우              
        if profit_flag==0: 
            if X1[n*4]<0.70 or X2[n*4]<0.70:
                value=min(X1[n*4]*100,X2[n*4]*100)/pow(1+r,2)
                case=6
            elif X1[n*4]>=0.70 and X2[n*4]>=0.70:
                value=100*pow(1+0.125,2)/pow(1+r,2)
                case=4
        value_list[j]=value
        case_list[j]=case
    value=np.mean(value_list)
    err=np.std(value_list)/np.sqrt(M)
    unique, counts = np.unique(case_list, return_counts=True)
    counts=counts/M
    prob=dict(zip(unique, counts))
    return value,err,prob #value=ELS의 가격 payoff의 현재가치
from datetime import datetime
r=0.03
q1=0.0
q2=0.0#배당률
sigma1=0.3
sigma2=0.4
rho=0.2
M=10000
n=8#6개월을 8개로 나눔
t1=datetime.now()
value,err,prob=ELS(r,q1,q2,sigma1,sigma2,rho,M,n)
t2=datetime.now()
print('value = {:.2f}, err = {:.2f}'.format(value,err))
for i in range(6):
    try:
        print('prob[{:d}] = {:{width}.2%}'.format(i+1,prob[i+1],width=6))
    except KeyError:
        print('prob[{:d}] = {:{width}.2%}'.format(i+1,0,width=6))
print('Total sum of prob = {:.0%}'.format(sum(prob.values())))
print('Total computing time = {:f} seconds'.format((t2-t1).total_seconds()))

value = 96.25, err = 0.24
prob[1] = 56.33%
prob[2] = 10.51%
prob[3] =  3.79%
prob[4] =  9.96%
prob[5] =  0.32%
prob[6] = 19.09%
Total sum of prob = 100%
Total computing time = 7.757619 seconds
